In [1]:
#purpose is to take HST images and identify extended sources (namely HH objects)

#just to check python version - should be 3.7.4
from platform import python_version
print(python_version())

#importing libraries
from astropy.io import fits
from astropy.convolution import convolve, Gaussian2DKernel, Box2DKernel
from astropy.nddata import Cutout2D
from astropy.wcs import WCS
from reproject import reproject_exact  #a package that can be added to astropy using anaconda or pip (see their docs pg)
from reproject import reproject_interp

import glob
import itertools
import matplotlib 
matplotlib.use('Agg') #invokved b/c just plain matplotlib was insufficient
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys

3.8.5


In [2]:
# #finding the path to every fits images in a directory
def im_name_finder(path, file_type):
    #Using glob (it's a unix command similar to ls)
    #WARNING: using recursive=True...depending how many images you use this could be very slow, it's recommended not to have too many subfolders
    #if needed, some example code is commented towards the latter half of this code that could help make an alternative
    all_names = glob.glob(path, recursive=True)

    #IMPORTANT: Using "fit" here because it is inclusive of both fits and FIT...some files end in "FIT" and need to be included
    #can edit this accordingly or edit your file names
    #using s.lower() includes uppercase extensions like FIT
    im_names = [s for s in all_names if 'fit' in s.lower()]

    return im_names

# #EX: grabbing all the fits image paths in a directory, #so they can be looped through and their data opened
# #set your path to some directory with images # (the images can be in subdirectories)
#the spitzer images
path = '../../n1333_photometry_ds9.bck.dir/**' # #using ** will grab all files even in subdirectories WARNING takes longer
im_names_spitz = im_name_finder(path, 'fit') #im_finder is basically glob.glob
im_names_spitz = [i.replace('\\', '/') for i in im_names_spitz]
im_names_spitz.remove('../../n1333_photometry_ds9.bck.dir/Frame20/n1333_band2_300_300_residual_hdrswap.fits')


#the hubble images
files_units = ['../../ngc1333_fits/126build_shift_2_drz.fits', 
                '../../ngc1333_fits/128build_shift_2_drz.fits', 
                '../../ngc1333_fits/164build_shift_2_drz.fits']

files_data = ['../../ngc1333_fits/126build_shift_2_drz.fits', 
                '../../ngc1333_fits/128build_shift_2_drz.fits', 
                '../../ngc1333_fits/164build_shift_2_drz.fits']

#initializing some lists to be used
hdu_pix_list = []
hdu_pixtorad_list = []
hdu_fnu_list = []
hdu_flam_list = []
hdu_bw_list = []
hdu_data_list = []
hdu_header_list = []
# we ignore throughput since it is included (?) in the other conversion values I'm told
# throughput_list = [1., 1., 1., 1., 1., 1.] # [0.242, 1., 0.246, 0.496, 0.521, 0.470] #also has to be done by hand, not in the headers?

files_units = im_names_spitz + files_units
files_data = im_names_spitz + files_data

#opening data and headers
hdu_list_units = [fits.open(i) for i in files_units]
hdu_list = [fits.open(i) for i in files_data]

    
#I'm using count here just to point to specific indices that I've set up...unfortunately some have different headers...
#the only diff between the if and else cases are the indexing of the hdu's, some need 1 and some need 0
#I've tried to group it for convience, so the the first two have the same headers, the last 3 have the same headers
count = 0
for (hdu_units,hdu_data) in zip(hdu_list_units, hdu_list):
    if count < len(hdu_list)-3: #the spitzer ones
        #reading in conversions
        print(count, files_units[count])
        hdu_pix_list.append(hdu_units[0].header['CDELT2'])  #CDELT2 is the keyword for Spitzer images, in arcsec
        hdu_pixtorad_list.append(hdu_pix_list[count] * np.pi / 180.) #convert to radians
        # hdu_fnu_list.append(hdu_units[0].header['PHOTFNU'])
        # hdu_flam_list.append(hdu_units[0].header['PHOTFLAM'])
#         hdu_bw_list.append(hdu_units[0].header['PHOTBW'])

        #reading in datafor general use  and header for wcs
        #the spiter data is in MJy / sr, so let's convert out the MJy to Flam units, times by bw/calscale from earlier
        #this gets units of erg/cm^2/sec - we also times by 1 rad^2 - 1 sr!  
        hdu_data_list.append(hdu_data[0].data * 1e6 * 1e-23 * hdu_pixtorad_list[count]**2.)  
        hdu_header_list.append(hdu_data[0].header)
        
    else: #the hubble ones
        #reading in conversions
        hdu_pix_list.append(hdu_units[0].header['D001SCAL'])  #D001SCAL is the keyword for Hubble images, in sr
        hdu_pixtorad_list.append(hdu_pix_list[count] / 206265.)
        # hdu_fnu_list.append(hdu_units[0].header['PHOTFNU'])
        hdu_flam_list.append(hdu_units[0].header['PHOTFLAM'])
        hdu_bw_list.append(hdu_units[0].header['PHOTBW'])

        #reading in data for general use  and header for wcs
        #converting by times by flam * bw from e-/sec...should get units of erg/cm^2/sec as above
        hdu_data_list.append(hdu_data[1].data * hdu_units[0].header['PHOTFLAM'] * hdu_units[0].header['PHOTBW'])
        print(hdu_units[0].header['PHOTFLAM'], hdu_units[0].header['PHOTBW'])
        hdu_header_list.append(hdu_data[1].header)

    count += 1

print('loaded data!')

0 ../../n1333_photometry_ds9.bck.dir/Frame1/n1333_lh_3_FeII_flag400_2_map_fixed.FIT
1 ../../n1333_photometry_ds9.bck.dir/Frame10/n1333_sh_1_S32_map.fits
2 ../../n1333_photometry_ds9.bck.dir/Frame11/n1333_sh_NeII_2_map.fits
3 ../../n1333_photometry_ds9.bck.dir/Frame12/n1333_sh_S2_2_map.fits
4 ../../n1333_photometry_ds9.bck.dir/Frame13/n1333_sl1_NeII_bksub_trim_2_map.fits
5 ../../n1333_photometry_ds9.bck.dir/Frame14/n1333_sl1_S2_bksub_trim_2_map.fits
6 ../../n1333_photometry_ds9.bck.dir/Frame15/n1333_sl1_S3_bksub_trim_2_map_fixed1.FIT
7 ../../n1333_photometry_ds9.bck.dir/Frame16/n1333_sl1_S4_bksub_trim_3_map.fits
8 ../../n1333_photometry_ds9.bck.dir/Frame17/n1333_sl2_S5_bksub_trim_2_map.fits
9 ../../n1333_photometry_ds9.bck.dir/Frame18/n1333_sl2_S6_bksub_trim_3_map.fits
10 ../../n1333_photometry_ds9.bck.dir/Frame19/n1333_sl2_S7_bksub_trim_2_map.fits
11 ../../n1333_photometry_ds9.bck.dir/Frame2/n1333_lh_3_S0_flag400_2_map_fixed.FIT
12 ../../n1333_photometry_ds9.bck.dir/Frame3/n1333_lh_3_S

In [3]:
%matplotlib inline
hdu_list

[[<astropy.io.fits.hdu.image.PrimaryHDU object at 0x0000013273BD3FA0>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x0000013273ED2AC0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x00000132741F4220>, <astropy.io.fits.hdu.image.ImageHDU object at 0x0000013271A27460>, <astropy.io.fits.hdu.image.ImageHDU object at 0x000001326DDB7550>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001326DDC2430>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x0000013273F977F0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x00000132742082E0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x000001327360E250>, <astropy.io.fits.hdu.image.ImageHDU object at 0x000001327360E430>, <astropy.io.fits.hdu.table.BinTableHDU object at 0x000001327360E670>],
 [<astropy.io.fits.hdu.image.PrimaryHDU object at 0x00000132740A3520>, <astropy.io.fits.hdu.image.ImageHDU object at 0x00000132742080A0>, <astropy.io.fits.hdu.image.ImageHDU object at 0x0000013274214D30>, <astropy.io.fits.hdu.image.ImageHDU o

In [ ]:
flux- or snr based conditional on astropy segmentation maps or surrounding local maxes

https://photutils.readthedocs.io/en/stable/isophote.html
    
https://stackoverflow.com/questions/5666056/matplotlib-extracting-data-from-contour-lines

https://stackoverflow.com/questions/36399381/whats-the-fastest-way-of-checking-if-a-point-is-inside-a-polygon-in-python